In [2]:
from pathlib import Path
import sys, importlib.util

def add_project_root(marker="src", max_up=6):
    """현재 CWD에서 위로 올라가며 'src' 폴더가 보이는 지점을 sys.path에 추가"""
    p = Path.cwd()
    for _ in range(max_up):
        if (p/marker).exists():
            p_str = str(p)
            if p_str not in sys.path:
                sys.path.insert(0, p_str)  # 최우선
            return p
        p = p.parent
    raise RuntimeError(f"'{marker}' not found within {max_up} parents from {Path.cwd()}")

root = add_project_root()  # 보통 /home/.../Projects/minervini
print("USING ROOT:", root)
print("ROOT/src exists?:", (root/"src").exists())
print("find_spec('src'):", importlib.util.find_spec("src"))

USING ROOT: /home/brighter87/Projects/minervini
ROOT/src exists?: True
find_spec('src'): ModuleSpec(name='src', loader=None, submodule_search_locations=_NamespacePath(['/home/brighter87/Projects/minervini/src']))


In [3]:
import mplfinance as mpf
from src.analysis.prepare import get_ohlc_ma_rs_for_analysis

df = get_ohlc_ma_rs_for_analysis()
df = df.set_index("date")

# OHLC 칼럼 이름을 mplfinance에 맞추기
df = df.rename(columns={
    "o":"Open", "h":"High", "l":"Low", "c":"Close", "v":"Volume"
})

# apds = [
#     mpf.make_addplot(df['ma50'],  color='#FFA500', width=1.2),
#     mpf.make_addplot(df['ma150'], color='#8A2BE2', width=1.2),
#     mpf.make_addplot(df['ma200'], color='#228B22', width=1.2),
# ]

# mpf.plot(
#     df,
#     type='candle',
#     addplot=apds,
#     volume=True,
#     style='yahoo',
#     ylabel='Price',
#     ylabel_lower='Volume',
#     tight_layout=True,
#     figscale=1.2,
#     figratio=(16,9),
#     # legend 켜기
#     returnfig=True
# )

In [8]:
df.columns

Index(['T', 'Volume', 'vw', 'Open', 'Close', 'High', 'Low', 't', 'n', 'ma50',
       'ma150', 'ma200', 'index_ret_50', 'index_ret_150', 'index_ret_200',
       'stock_ret_50', 'RS_50', 'stock_ret_150', 'RS_150', 'stock_ret_200',
       'RS_200', 'RS_50_pct', 'RS_150_pct', 'RS_200_pct', 'RS_pct_min',
       'RS_pct_mean'],
      dtype='object')

In [26]:
ticker = "flnc"
ticker = ticker.upper()
df_ticker = df.loc[df["T"] == ticker]
df_ticker["RS_200_pct"]

date
2022-06-14         NaN
2022-06-15         NaN
2022-06-16         NaN
2022-06-17         NaN
2022-06-21         NaN
                ...   
2025-09-19    0.076227
2025-09-22    0.093488
2025-09-23    0.091236
2025-09-24    0.125732
2025-09-25    0.150614
Name: RS_200_pct, Length: 824, dtype: float64

In [28]:
c1 = df['Close'] > df['ma50']
c2 = df['Close'] > df['ma150']
c3 = df['Close'] > df['ma200']
c4 = df['ma50'] > df['ma150']
c5 = df['ma150'] > df['ma200']
c6 = df['RS_50_pct'] > 0.7
c6 = df['RS_150_pct'] > 0.7
c6 = df['RS_200_pct'] > 0.7

df

,T,Volume,vw,Open,Close,High,Low,t,n,ma50,...,RS_50,stock_ret_150,RS_150,stock_ret_200,RS_200,RS_50_pct,RS_150_pct,RS_200_pct,RS_pct_min,RS_pct_mean
date,,,,,,,,,,,,,,,,,,,,,
2022-06-14,A,1314047.0,115.1755,115.99,116.01,116.7000,113.250,1655236800000,27280.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-15,A,1886299.0,116.1269,117.24,116.47,117.7700,113.770,1655323200000,35363.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-16,A,1601588.0,114.3379,113.80,114.96,115.0800,113.360,1655409600000,30606.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-17,A,2681452.0,113.6622,114.77,112.71,116.7300,112.670,1655496000000,36559.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-21,A,1588225.0,114.1450,114.41,113.88,115.5700,113.760,1655841600000,23893.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-19,ZYXI,89921.0,1.5823,1.58,1.55,1.6050,1.550,1758312000000,992.0,1.7940,...,0.624552,0.204756,0.183079,0.186298,0.158444,0.041524,0.014761,0.020088,0.014761,0.025458
2025-09-22,ZYXI,77296.0,1.5793,1.55,1.58,1.6100,1.550,1758571200000,760.0,1.7808,...,0.649135,0.209549,0.187261,0.192214,0.164474,0.044341,0.014376,0.020242,0.014376,0.026320
2025-09-23,ZYXI,118350.0,1.5481,1.60,1.53,1.6000,1.495,1758657600000,812.0,1.7692,...,0.674220,0.199739,0.180096,0.188192,0.162606,0.051716,0.013470,0.019313,0.013470,0.028166
